In [44]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from os import name
from parse import load_dataframes
from scipy import sparse
import scipy



In [45]:
def get_user_store(dataframes):
    stores_reviews = pd.merge(
        dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store"
    )
    
    user_list = list(set(stores_reviews["user"].values.tolist()))
    user_list.sort()
    store_list = list(set(stores_reviews["store_name"].values.tolist()))
    df = pd.DataFrame(data=np.nan, index=user_list, columns=store_list)
    
    
    user_group = stores_reviews.sort_values(by='user').groupby(["user", "store_name"]).mean().loc[:, "score"]

    for i, score in user_group.items():
        user, store_name = i
        df.loc[user,  store_name] = score
    sdf = df.astype(pd.SparseDtype("int", np.nan))
    
    print(df.info())
    print("---------------------")
    print(sdf.info())

In [50]:
def get_user_category(dataframes):
    # stores_reviews = pd.merge(
    #     dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store"
    # )

    # user_list = list(set(stores_reviews["user"].values.tolist()))
    # user_list.sort()
    # category_list = stores_reviews["category"].str.split("|")
    # category_list = category_list.explode("category")
    # df = pd.DataFrame(data=np.nan, index=user_list, columns=category_list)
    # user_group = 


    # 카테고리 | 로 구분되어 있는거를 리스트로 넣어줘야한다
    stores = dataframes["stores"][["id","category", "review_cnt"]]
    # category 가 없거나 review가 없는 음식점 제거.
    stores = stores[(stores["category"] != "") & (stores["review_cnt"] > 0)]
    # 구분자("|") 로 구분되어 있는 category를 리스트 형태로 변환한다.
    stores["category"] = stores["category"].str.split("|")
    # category를 기준으로 row를 분리함
    stores = stores.explode("category")

    reviews = dataframes["reviews"][["store", "user", "score"]]
    stores_reviews = pd.merge(stores, reviews, left_on="id", right_on="store")
    grouped_stores_reviews = stores_reviews.groupby(["user", "category"])["score"].mean().reset_index(name="score")

    uc_pivot_table = grouped_stores_reviews.pivot_table(index="user", columns="category", values="score")
#     print(uc_pivot_table.info())
#     print("---------------------------------------")
    uc_pivot_sparsetype = uc_pivot_table.astype(pd.SparseDtype("float", np.nan))
    
    csr_matrix = scipy.sparse.csr_matrix(uc_pivot_sparsetype)
#     print(uc_pivot_sparsetype.info())
    return csr_matrix
    

In [51]:
def main():
#     set_config()
    data = load_dataframes()
    # show_store_categories_graph(data)
    # show_store_review_distribution_graph(data)
    # show_store_average_ratings_graph(data)
    # show_user_review_distribution_graph(data)
    # show_user_age_gender_distribution_graph(data)
    # show_stores_distribution_graph(data)
#     get_user_store(data)
    get_user_category(data)

if __name__ == "__main__":
    main()
